<a href="https://colab.research.google.com/github/krishnavikas-7/INFO-5731/blob/main/INFO5731_Assignment_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Two**

In this assignment, you will try to gather text data from open data source via web scraping or API. After that you need to clean the text data and syntactic analysis of the data.

# **Question 1**

(20 points). Write a python program to collect text data from **either of the following sources** and save the data into a **csv file**:

(1) Collect all the customer reviews of the product [Apple iPhone 11](https://www.amazon.com/Apple-iPhone-11-64GB-Unlocked/dp/B07ZPKF8RG/ref=sr_1_13?dchild=1&keywords=iphone+12&qid=1631721363&sr=8-13) on amazon.

(2) Collect all User Reviews of the film [Shang-Chi and the Legend of the Ten Rings](https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3) from IMDB.

(3) Collect all the abstracts research papers by using the query [smart health](https://citeseerx.ist.psu.edu/search?q=natural+language+processing&submit.x=0&submit.y=0&sort=rlv&t=doc) from CiteSeerX.

(4) Collect the top 10,000 tweets by using hashtag ["#blacklivesmatter"](https://twitter.com/hashtag/blacklivesmatter) from Twitter with the following requirements: Location (Texas), Time frame (2021-01-01 to 2021-09-01). 


In [ ]:
# Write your code here


#importing Libraries
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup as BS
import re
import numpy as np
import pandas as pd

#function calling
def get_tablelist(soup):
    reviews = soup.find_all("div", {"class": "lister-item-content"})

    review_data = []

    for review in reviews:

        try:
            review_content = review.find("div", {"class": "text show-more__control"}).text.strip()
        except:
            review_content = review.find("div", {"class": "text show-more__control clickable"}).text.strip()

        review_data.append(review_content)
    return review_data

b_url = 'https://www.imdb.com/'
url = 'https://www.imdb.com/title/tt9376612/reviews?ref_=tt_sa_3'
res = requests.get(url)
soup = BS(res.text, 'html.parser')
res.encoding = 'utf-8'
allTablelist=[]
tablelist = get_tablelist(soup)
for i in tablelist:
  allTablelist.append(i)

load_more = soup.select(".load-more-data")
flag = True
if len(load_more):
    ajaxurl = load_more[0]['data-ajaxurl']
    b_url = b_url + ajaxurl + "?ref_=undefined&paginationKey="
    try:
        key = load_more[0]['data-key']
    except KeyError:
        flag = False
else:
    flag = False

while flag:
    url = b_url + key
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BS(res.text, 'html.parser')
    tablelist2 = get_tablelist(soup)
    if len(allTablelist) == 1235:
      break
    for i in tablelist2:
        allTablelist.append(i)
    load_more = soup.select(".load-more-data")
    if len(load_more):
        key = load_more[0]['data-key']
    else:
        flag = False

data = np.array(allTablelist)
data = pd.DataFrame(data, columns=['movie_Reviews'])
data.shape
data.to_csv('imdb_reviews.csv')
data




,movie_Reviews
0,I'll start by saying that if you're looking fo...
1,After 10 years of almost every movie being.arm...
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20..."
3,Perfect Fantasy film to watch with full family...
4,Keeping it short. This movie had it all. Great...
...,...
1245,Shang-Chi and the Legend of the Ten Rings (202...
1246,I walked into this film excited but slightly k...
1247,"Not bad, but not great. It was kinda like a Sp..."
1248,"At first, it felt like I am watching not a MCU..."


In [ ]:
data.to_csv('shangchi_marvel_reviews.csv', index=False)

# **Question 2**

(30 points) Here is a [legal case](https://github.com/unt-iialab/info5731-fall2021/blob/main/assignments/01-05-1%20%20Adams%20v%20Tanner.txt) we collected from westlaw, please follow the steps we mentioned in lesson 5 to clean the data:



(1) Basic feature extraction using text data

*   Number of sentences
*   Number of words
*   Number of characters
*   Average word length
*   Number of stopwords
*   Number of special characters
*   Number of numerics
*   Number of uppercase words

(2) Basic Text Pre-processing of text data

*   Lower casing
*   Punctuation removal
*   Stopwords removal
*   Frequent words removal
*   Rare words removal
*   Spelling correction
*   Tokenization
*   Stemming
*   Lemmatization

(3) Save all the **clean sentences** to a **csv file** (one column, each raw is a sentence) after finishing all the steps above.


(4) Advance Text Processing

*   Calculate the term frequency of all the terms.
*   Print out top 10 1-gram, top 10 2-grams, and top 10 3-grams terms as features.


**(1) Basic feature extraction using text data**

In [ ]:
#Number of characters
data['num_char'] = data['movie_Reviews'].str.len()
data[['movie_Reviews','num_char']].head()

,movie_Reviews,num_char
0,I'll start by saying that if you're looking fo...,1263
1,After 10 years of almost every movie being.arm...,597
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",5280
3,Perfect Fantasy film to watch with full family...,83
4,Keeping it short. This movie had it all. Great...,590


In [ ]:
#Number of words
data['word_count'] = data['movie_Reviews'].apply(lambda x: len(str(x).split(" ")))
data[['movie_Reviews','word_count']].head()

,movie_Reviews,word_count
0,I'll start by saying that if you're looking fo...,227
1,After 10 years of almost every movie being.arm...,105
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",927
3,Perfect Fantasy film to watch with full family...,15
4,Keeping it short. This movie had it all. Great...,102


In [ ]:
#average word length
def avg_word_length(sent):
  words = sent.split()
  return (sum(len(word) for word in words)/(len(words)+1))

data['avg_word_length'] = data['movie_Reviews'].apply(lambda x: avg_word_length(x))
data[['movie_Reviews','avg_word_length']].head()

,movie_Reviews,avg_word_length
0,I'll start by saying that if you're looking fo...,4.548246
1,After 10 years of almost every movie being.arm...,4.650943
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",4.691810
3,Perfect Fantasy film to watch with full family...,4.312500
4,Keeping it short. This movie had it all. Great...,4.747573


In [ ]:
#Number of stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['stopwords'] = data['movie_Reviews'].apply(lambda x: len([x for x in x.split() if x in stop]))
data[['movie_Reviews','stopwords']].head()

,movie_Reviews,stopwords
0,I'll start by saying that if you're looking fo...,102
1,After 10 years of almost every movie being.arm...,41
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",228
3,Perfect Fantasy film to watch with full family...,5
4,Keeping it short. This movie had it all. Great...,40


In [ ]:
#Number of special characters
data['hastags'] = data['movie_Reviews'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
data[['movie_Reviews','hastags']].head()

,movie_Reviews,hastags
0,I'll start by saying that if you're looking fo...,0
1,After 10 years of almost every movie being.arm...,0
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",0
3,Perfect Fantasy film to watch with full family...,0
4,Keeping it short. This movie had it all. Great...,0


In [ ]:
#Number of numerics
data['numerics'] = data['movie_Reviews'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
data[['movie_Reviews','numerics']].head()


,movie_Reviews,numerics
0,I'll start by saying that if you're looking fo...,2
1,After 10 years of almost every movie being.arm...,2
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",4
3,Perfect Fantasy film to watch with full family...,0
4,Keeping it short. This movie had it all. Great...,0


In [ ]:
#Number of uppercase words
data['upper'] = data['movie_Reviews'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
data[['movie_Reviews','upper']].head()

,movie_Reviews,upper
0,I'll start by saying that if you're looking fo...,5
1,After 10 years of almost every movie being.arm...,0
2,"A -BIG- Screen Mini Review. Viewed Sept.05, 20...",26
3,Perfect Fantasy film to watch with full family...,0
4,Keeping it short. This movie had it all. Great...,3


**(2) Basic Text Pre-processing of text data**

In [ ]:
# Write your code here
#Lower Casing
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
data['movie_Reviews'].head()





0    i'll start by saying that if you're looking fo...
1    after 10 years of almost every movie being.arm...
2    a -big- screen mini review. viewed sept.05, 20...
3    perfect fantasy film to watch with full family...
4    keeping it short. this movie had it all. great...
Name: movie_Reviews, dtype: object

In [ ]:
# punctuation removal
data['movie_Reviews'] = data['movie_Reviews'].str.replace('[^\w\s]','')
data['movie_Reviews'].head()

0    ill start by saying that if youre looking for ...
1    after 10 years of almost every movie beingarme...
2    a big screen mini review viewed sept05 2021___...
3    perfect fantasy film to watch with full family...
4    keeping it short this movie had it all great s...
Name: movie_Reviews, dtype: object

In [ ]:
#remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data['movie_Reviews'].head()

0    ill start saying youre looking great story you...
1    10 years almost every movie beingarmed combat ...
2    big screen mini review viewed sept05 2021_____...
3    perfect fantasy film watch full family cant mi...
4    keeping short movie great story writing direct...
Name: movie_Reviews, dtype: object

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91773\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#remove special characters
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
#remove special characters
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join(x.strip(string.punctuation) for x in x.split()))  
data['movie_Reviews']

0       ill start saying youre looking great story you...
1       10 years almost every movie beingarmed combat ...
2       big screen mini review viewed sept05 2021_____...
3       perfect fantasy film watch full family cant mi...
4       keeping short movie great story writing direct...
                              ...                        
1245    shangchi legend ten rings 2021 brief review ye...
1246    walked film excited slightly keeping expectati...
1247    bad great kinda like spiderman movie really ac...
1248    first felt like watching mcu movie felt differ...
1249    im strong mcu fan im one enjoy almost everythi...
Name: movie_Reviews, Length: 1250, dtype: object

In [ ]:
#Tokkenization
from nltk.tokenize import word_tokenize
pos = []
for sentence in data['movie_Reviews']:
  text = word_tokenize(sentence)
  pos.append(nltk.pos_tag(text))
pos


[[('ill', 'JJ'),
  ('start', 'VB'),
  ('saying', 'VBG'),
  ('youre', 'NN'),
  ('looking', 'VBG'),
  ('great', 'JJ'),
  ('story', 'NN'),
  ('youll', 'NN'),
  ('disappointed', 'VBD'),
  ('shangchi', 'JJ'),
  ('pretty', 'RB'),
  ('standard', 'JJ'),
  ('heros', 'NNS'),
  ('journey', 'NN'),
  ('core', 'NN'),
  ('shame', 'NN'),
  ('story', 'NN'),
  ('could', 'MD'),
  ('inspired', 'VB'),
  ('house', 'NN'),
  ('flying', 'VBG'),
  ('daggers', 'NNS'),
  ('wuxia', 'VBP'),
  ('titlesso', 'IN'),
  ('rating', 'NN'),
  ('high', 'JJ'),
  ('story', 'NN'),
  ('falls', 'VBZ'),
  ('everything', 'NN'),
  ('else', 'RB'),
  ('excelsshangchi', 'VBZ'),
  ('visually', 'RB'),
  ('spectacular', 'JJ'),
  ('real', 'JJ'),
  ('treat', 'NN'),
  ('eyes', 'NNS'),
  ('visuals', 'NNS'),
  ('incredibly', 'RB'),
  ('distinct', 'JJ'),
  ('mcu', 'NN'),
  ('regards', 'NNS'),
  ('aesthetics', 'NNS'),
  ('black', 'JJ'),
  ('panther', 'NN'),
  ('walked', 'VBD'),
  ('shangchi', 'NNS'),
  ('could', 'MD'),
  ('run', 'VB'),
  ('marat

In [ ]:
!pip install textblob

In [ ]:
#spelling Correction
from textblob import TextBlob
data['movie_Reviews'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    ill start saying your looking great story you ...
1    10 years almost every movie beingarmed combat ...
2    big screen mind review viewed septic 2021_____...
3    perfect fantasy film watch full family can mis...
4    keeping short movie great story writing direct...
Name: movie_Reviews, dtype: object

In [ ]:
#stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
data['movie_Reviews'][:5].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

0    ill start say your look great stori youll disa...
1    10 year almost everi movi beingarm combat one ...
2    big screen mini review view sept05 2021_______...
3    perfect fantasi film watch full famili cant mi...
4    keep short movi great stori write direct act v...
Name: movie_Reviews, dtype: object

In [ ]:
#lemmatization
from textblob import Word
nltk.download('wordnet')
data['movie_Reviews'] = data['movie_Reviews'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
data['movie_Reviews'].head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91773\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0    ill start saying youre looking great story you...
1    10 year almost every movie beingarmed combat o...
2    big screen mini review viewed sept05 2021_____...
3    perfect fantasy film watch full family cant mi...
4    keeping short movie great story writing direct...
Name: movie_Reviews, dtype: object

In [ ]:
#(3) Save all the clean sentences to a csv file (one column, each raw is a sentence) after finishing all the steps above.
data.to_csv('shangchi_marvel_reviews.csv', index=False)

In [ ]:
# (4) Advance Text Processing
#Calculate the term frequency of all the terms.
data_f = (data['movie_Reviews'][0:10]).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0).reset_index()
data_f.columns = ['words','term frequency']
data_f

,words,term frequency
0,fight,8.0
1,story,12.0
2,scene,6.0
3,shangchi,9.0
4,one,11.0
...,...,...
633,found,1.0
634,mediocre,1.0
635,streaming,1.0
636,connect,1.0


# **Question 3**

(30 points). Write a python program to conduct **syntax and structure analysis** of the clean text you just saved above. The syntax and structure analysis includes: 

(1) Parts of Speech (POS) Tagging: Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) Named Entity Recognition: Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
# Write your code here
nltk.download('averaged_perceptron_tagger')




[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\91773\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91773\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# 3(1) Parts of speech(POS) Tagging
from nltk import word_tokenize, pos_tag, pos_tag_sents
data['POSTags'] =  pos_tag_sents( data['movie_Reviews'].apply(word_tokenize).tolist() )
print(data)

pos = data['POSTags'].to_list()
pos
nouns = 0
for x in pos:
  for a,b in x:
    if(b.startswith('N')):
       nouns+=1
output = "nouns:{}".format(nouns)
print(output)
verb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('V')):
       verb+=1
output1 = "verb:{}".format(verb)
print(output1)
adverb = 0
for x in pos:
  for a,b in x:
    if(b.startswith('R')):
       adverb+=1
output2 = "adverb:{}".format(adverb)
print(output2)
adj = 0
for x in pos:
  for a,b in x:
    if(b.startswith('J')):
       adj+=1
output3 = "adj:{}".format(adj)
print(output3)

                                          movie_Reviews  num_char  word_count  \
0     ill start saying youre looking great story you...      1263         227   
1     10 year almost every movie beingarmed combat o...       597         105   
2     big screen mini review viewed sept05 2021_____...      5280         927   
3     perfect fantasy film watch full family cant mi...        83          15   
4     keeping short movie great story writing direct...       590         102   
...                                                 ...       ...         ...   
1245  shangchi legend ten ring 2021 brief review yet...      2512         418   
1246  walked film excited slightly keeping expectati...      1820         324   
1247  bad great kinda like spiderman movie really ac...       576         102   
1248  first felt like watching mcu movie felt differ...      1127         199   
1249  im strong mcu fan im one enjoy almost everythi...      1157         215   

      avg_word_length  stop

**2)** **Constituency Parsing and Dependency Parsing: print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.**


**Consituency parsing** - displays syntactic structure where the words are organized into nested constituents. Both are helpful in word processing systems and grammar correction.


**Dependency parsing** - displays grammatical structure where each word represents a node and followed by links to its dependents. It displays the relation between words.



In [ ]:
#entity recognition
!pip install spacy

In [ ]:
!pip3 install en_core_web_sm
!python -m spacy download en

  Could not find a version that satisfies the requirement en_core_web_sm (from versions: )
No matching distribution found for en_core_web_sm
You are using pip version 10.0.1, however version 21.2.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# 3(2) Name entity Recognition
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = spacy.load("en_core_web_sm")
data['movie_Reviews']
list = data['movie_Reviews'].tolist()
list[0]
for x in list:
   doc = (nlp(x))
   print([(X.text, X.label_) for X in doc.ents])

[('one', 'CARDINAL'), ('5 10 minute', 'TIME'), ('first', 'ORDINAL')]
[('10 year', 'DATE'), ('one', 'CARDINAL'), ('4', 'CARDINAL'), ('liu shangchi', 'PERSON')]
[('1', 'CARDINAL'), ('first', 'ORDINAL'), ('ten', 'CARDINAL'), ('malaysian', 'NORP'), ('michelle yeoh', 'PERSON'), ('one', 'CARDINAL'), ('first', 'ORDINAL'), ('ten', 'CARDINAL'), ('year 2008s', 'DATE'), ('first', 'ORDINAL'), ('usa long', 'EVENT'), ('afghanistan', 'GPE'), ('islam', 'ORG'), ('year', 'DATE'), ('arab', 'NORP'), ('100', 'CARDINAL'), ('chinese', 'NORP'), ('ten', 'CARDINAL'), ('destin daniel cretton', 'PERSON'), ('5th', 'ORDINAL'), ('10', 'CARDINAL'), ('stan lee', 'PERSON'), ('feb 1964', 'DATE'), ('four', 'CARDINAL'), ('simu liu', 'PERSON'), ('katy awkwafina', 'PERSON'), ('tony leung menger', 'PERSON'), ('fala chen michelle yeoh', 'PERSON'), ('one', 'CARDINAL'), ('ben kingsley perform', 'PRODUCT'), ('950 mark 10', 'MONEY')]
[]
[('directionim huge old', 'ORG'), ('jackie chan', 'PERSON'), ('today', 'DATE'), ('last night',

# **Question 4**

(20 points) Python Regular Expression.

(1) Write a Python program to remove leading zeros from an IP address.

ip = "260.08.094.109"


(2) Write a Python Program to extract all the years from the following sentence.

sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."

In [ ]:
# (1) Write a Python program to remove leading zeros from an IP address.

#ip = "260.08.094.109"
#function for zero removing
def removingzeros(ip):
    latest_ip = re.sub(r'\b0+(\d)', r'\1', ip)
      
    return latest_ip 
#driver code
# example
ip ="005.700.004.008"
print(removingzeros(ip))

5.700.4.8


In [ ]:
#(2) Write a Python Program to extract all the years from the following sentence.sentence = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
import re
 
# entering string 
new_string = "The 2010s were a dramatic decade, filled with ups and downs, more than 1000 stroies have happened. As the decade comes to a close, Insider took a look back at some of the biggest headline-grabbing stories, from 2010 to 2019. The result was 119 news stories that ranged from the heartwarming rescue of a Thai boys' soccer team from a flooded cave to the divisive election of President Donald Trump."
# extracting words from string
res = re.findall(r'\b\d{4}\b', new_string)
 
# printing the final output
print ("words are : " +  str(res))

words are : ['1000', '2010', '2019']
